In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Concatenate, Reshape
from tensorflow.keras import Input
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train['binary_open_channels'] = [int(x != 0) for x in df_train.open_channels]
open_channels = df_train.open_channels
df_train = pd.get_dummies(df_train, prefix = 'oc',columns=['open_channels'])
df_train['open_channels'] = open_channels
df_train

,time,signal,binary_open_channels,oc_0,oc_1,oc_2,oc_3,oc_4,oc_5,oc_6,oc_7,oc_8,oc_9,oc_10,open_channels
0,0.0001,-2.7600,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.0002,-2.8557,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.0003,-2.4074,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0.0004,-3.1404,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0.0005,-3.1525,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,499.9996,2.9194,1,0,0,0,0,0,0,0,1,0,0,0,7
4999996,499.9997,2.6980,1,0,0,0,0,0,0,0,1,0,0,0,7
4999997,499.9998,4.5164,1,0,0,0,0,0,0,0,0,1,0,0,8
4999998,499.9999,5.6397,1,0,0,0,0,0,0,0,0,0,1,0,9


In [3]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train)
categorical_output = df_train[['oc_0','oc_1','oc_2','oc_3','oc_4','oc_5','oc_6','oc_7','oc_8','oc_9','oc_10']]

data = np.zeros((len(signal), 12))
data[:, 0] = signal
data[:, 1:] = categorical_output
data = data.reshape((1000, 5000, 12))
np.random.shuffle(data)

x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1:].reshape((1000,5000,11))

In [4]:
np.shape(x),np.shape(y)

((1000, 5000, 1), (1000, 5000, 11))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .3)

In [6]:
inputs = Input(shape=(5000,1,))

layer1_a = LSTM(units=3, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_a = Dense(5,activation='relu')(layer1_a)

layer1_b = LSTM(units=5, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_b = Dense(8,activation='relu')(layer1_b)

layer1_c = SimpleRNN(units=3, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_c = Dense(5,activation='relu')(layer1_c)

layer1_d = SimpleRNN(units=5, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_d = Dense(8,activation='relu')(layer1_d)
concat = Concatenate()([layer2_a, layer2_b,layer2_c,layer2_d])

outputs = Dense(units=11, activation='softmax')(concat)

In [7]:
model = Model(inputs,outputs)
opt = Adam(learning_rate=.01,beta_1=0.95)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5000, 1)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 5000, 3)      60          input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 5000, 5)      140         input_1[0][0]                    
__________________________________________________________________________________________________
simple_rnn (SimpleRNN)          (None, 5000, 3)      15          input_1[0][0]                    
______________________________________________________________________________________________

In [6]:
model = tf.keras.models.load_model('../Models/multi_path_model_63.h5')

In [8]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 95s 136ms/sample - loss: 1.8049 - accuracy: 0.3429 - val_loss: 1.6030 - val_accuracy: 0.3875
Epoch 2/10
700/700 [==============================] - 94s 134ms/sample - loss: 1.4979 - accuracy: 0.4521 - val_loss: 1.4564 - val_accuracy: 0.4910
Epoch 3/10
700/700 [==============================] - 94s 134ms/sample - loss: 1.3797 - accuracy: 0.4872 - val_loss: 1.3914 - val_accuracy: 0.4608
Epoch 4/10
700/700 [==============================] - 94s 134ms/sample - loss: 1.3298 - accuracy: 0.4839 - val_loss: 1.3651 - val_accuracy: 0.4576
Epoch 5/10
700/700 [==============================] - 100s 143ms/sample - loss: 1.2984 - accuracy: 0.4941 - val_loss: 1.3379 - val_accuracy: 0.4771
Epoch 6/10
700/700 [==============================] - 96s 137ms/sample - loss: 1.2750 - accuracy: 0.4947 - val_loss: 1.3173 - val_accuracy: 0.4949
Epoch 7/10
700/700 [==============================] - 95s 136ms/sample 

In [9]:
model.fit(x_train, y_train, epochs=25, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/25
700/700 [==============================] - 121s 173ms/sample - loss: 1.0184 - accuracy: 0.6309 - val_loss: 1.0180 - val_accuracy: 0.6445
Epoch 2/25
700/700 [==============================] - 115s 164ms/sample - loss: 0.9810 - accuracy: 0.6412 - val_loss: 0.9930 - val_accuracy: 0.6489
Epoch 3/25
700/700 [==============================] - 117s 167ms/sample - loss: 0.9642 - accuracy: 0.6474 - val_loss: 0.9679 - val_accuracy: 0.6596
Epoch 4/25
700/700 [==============================] - 116s 166ms/sample - loss: 0.9521 - accuracy: 0.6475 - val_loss: 0.9523 - val_accuracy: 0.6565
Epoch 5/25
700/700 [==============================] - 118s 168ms/sample - loss: 0.9583 - accuracy: 0.6410 - val_loss: 0.9708 - val_accuracy: 0.6476
Epoch 6/25
700/700 [==============================] - 118s 168ms/sample - loss: 0.9473 - accuracy: 0.6444 - val_loss: 0.9410 - val_accuracy: 0.6612
Epoch 7/25
700/700 [==============================] - 118s 168ms/s

In [10]:
results = model.predict(x)

In [11]:
correct = 0
incorrect = 0
for i in range(len(results)):
    for j in range(len(results[0])):
        a = np.where(results[i][j] == np.amax(results[i][j]))[0]
        b = np.where(y[i][j] == np.amax(y[i][j]))[0]
        if a[0] == b[0]:
            correct += 1
        else:
            incorrect += 1
print(correct/(correct+incorrect))

0.7186488


In [12]:
model.save('../Models/multi_path_model_71.h5')